In [1]:
##試著使用鐵達尼號的例子，
##創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )
library(magrittr)
library(caret)
library(cattonum)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
str(train)
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')
train.num=train[,-col.factor]

## Age NA --> mean 
age.na.loc=which(is.na(train.num$Age)==T)
train.num$Age[age.na.loc]=mean(train.num$Age,na.rm = T)

##only keep categorical column
train.cate=train[,c(col.factor,1)]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## fill somethong in empty cell
for (d in 1:3) {
  levels(train.cate[,d])[levels(train.cate[,d])=='']='empty'
}

## 每一個類別feature都增加兩種群聚編碼 : mean, median 
mean.dat=catto_mean(train.cate,c('Sex','Cabin','Embarked'),response=Survived)
colnames(mean.dat)=c('Sex1','Cabin1','Embarked1')
median.dat=catto_median(train.cate,c('Sex','Cabin','Embarked'),response=Survived)
train.new=cbind(mean.dat[,-4],median.dat[-4],Survived=median.dat[,4],train.num[,-1])


##將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

#regression +numeric
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv1= train(as.factor(Survived)~., data=train.num, trControl=train_control, family=binomial,method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
acc1=reg.cv1$results[2]
kappa1=reg.cv1$results[3]

#xgb +numeric
t11=Sys.time()
xgb1=train(as.factor(Survived)~., data=train.num, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc1=mean(xgb1$results$Accuracy)
gb.kappa1=mean(xgb1$results$Kappa)
t12=Sys.time()
time.gb1=difftime(t11,t12,units='secs')


#regression+new
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv2= train(as.factor(Survived)~., data=train.new, trControl=train_control, family=binomial,method="glm")
t12=Sys.time()
time2=difftime(t11,t12,units='secs')
acc2=reg.cv2$results[2]
kappa2=reg.cv2$results[3]

#xgb+new
t11=Sys.time()
xgb2=train(as.factor(Survived)~., data=train.new, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc2=mean(xgb2$results$Accuracy)
gb.kappa2=mean(xgb2$results$Kappa)
t12=Sys.time()
time.gb2=difftime(t11,t12,units='secs')


result1=data.frame(type=c('reg','reg+mean+median','xgb','xgb+mean+median'),
                   acc=as.numeric(c(acc1,acc2,gb.acc1,gb.acc1)),
                   kappa=as.numeric(c(kappa1,kappa2,gb.kappa1,gb.kappa2)),
                   time=-c(time1,time2,time.gb1,time.gb2))
result1

Loading required package: lattice
Loading required package: ggplot2


ERROR: Error in library(cattonum): there is no package called 'cattonum'


### (1) 在regression 與 xgb中，增加群聚編碼:median 與 mean 確實大幅提升kappa